In [94]:
#os
import os

#ml
import numpy as np
import torch
import mmcv
import cv2
from mmdet.apis    import init_detector
from mmrotate.apis import inference_detector_by_patches
from dataclasses import asdict
import json

from geo_util import Pnt, Vehicle, VehicleExport, rbbox_to_poly

In [95]:

def process_results( result, **kwargs ):
    if isinstance(result, tuple):
        bbox_result, segm_result = result
        if isinstance( segm_result, tuple ):
            segm_result = segm_result[0]
    else:
        bbox_result, segm_result = result, None
    bboxes = np.vstack(bbox_result)
    labels = [
        np.full( bbox.shape[0], i, dtype=np.int32 )
        for i, bbox in enumerate(bbox_result)
    ]
    labels = np.concatenate( labels )
    #remove everything that isn't a small vehicle    
    mask = (labels == 4) | (labels == 5)
    labels = labels[mask]
    bboxes = bboxes[mask]
    print( f"Detected {len( bboxes )} vehicles." )
    reshaped = labels.reshape( -1, 1 )
    concat = np.concatenate( (bboxes, reshaped), axis=1 )
    a_entries = [Vehicle( r[0], r[1], r[2], r[3], r[4], r ) for r in concat]
    vas = [VehicleExport( Pnt( a.x, a.y ), a.width, a.height, a.theta, rbbox_to_poly( a.arr ).tolist(), str(i) ) for i, a in enumerate(a_entries)]
    return vas

In [96]:
class InitArgs:
    def __init__(self):
        self.config     = "mmrotate/configs/redet/redet_re50_refpn_1x_dota_ms_rr_le90.py"
        self.checkpoint = "model/redet_re50_fpn_1x_dota_ms_rr_le90-fc9217b5.pth"
        self.device     = 'cuda:0'

class Args:
    def __init__( self, batch_size: int, patch_size: int, patch_step: int, img: str ):
        self.img = img
        self.score_thr = 0.0025
        self.merge_iou_thr = 0.85
        self.img_ratios = [1.0]
        self.batch_size = batch_size
        self.patch_sizes = [patch_size]
        self.patch_steps = [patch_step]
        self.palette = 'dota'

torch.cuda.set_per_process_memory_fraction( 0.75 )

In [97]:
init_args = InitArgs()
try:
    model
except NameError:
    model = init_detector( init_args.config, init_args.checkpoint, device=init_args.device )

In [98]:
from pathlib import Path
import numpy as np
from typing import List, Tuple
from rtree import index
import torch
from mmrotate.core.bbox.transforms import norm_angle

def load_ground_truth(img_path: str) -> List[VehicleExport]:
    """Load ground truth from the corresponding .txt file"""
    img_path_obj = Path(img_path)
    gt_path = img_path_obj.with_suffix('.txt')
    
    if not gt_path.exists():
        print(f"Warning: Ground truth file {gt_path} not found")
        return []
    
    # Get image dimensions to convert normalized coordinates to absolute
    try:
        from PIL import Image
        with Image.open(img_path) as img:
            img_width, img_height = img.size
    except:
        # Fallback if image can't be opened
        img_width, img_height = 1024, 1024
        print(f"Warning: Could not get image dimensions for {img_path}, using 1024x1024")
    
    gt_vehicles = []
    with open(gt_path, 'r') as f:
        for i, line in enumerate(f):
            parts = line.strip().split()
            if len(parts) >= 6:
                # Parse C# YOLO format: class_id center_x_norm center_y_norm width_norm height_norm angle
                class_id = int(parts[0])
                cx_norm = float(parts[1])  # normalized center x
                cy_norm = float(parts[2])  # normalized center y
                w_norm = float(parts[3])   # normalized width
                h_norm = float(parts[4])   # normalized height
                angle = float(parts[5])    # angle in radians
                
                # Convert to le90 format (-π/2 to π/2)
                while angle > np.pi/2:
                    angle = angle - np.pi
                while angle < -np.pi/2:
                    angle = angle + np.pi
                
                # Convert normalized coordinates to absolute pixels
                cx_abs = cx_norm * img_width
                cy_abs = cy_norm * img_height
                w_abs = w_norm * img_width
                h_abs = h_norm * img_height
                
                # Create the array format that your Vehicle class expects
                # [x, y, width, height, angle, confidence, class_id]
                arr = np.array([cx_abs, cy_abs, w_abs, h_abs, angle, 1.0, class_id])
                poly = rbbox_to_poly(arr).tolist()
                
                gt_vehicle = VehicleExport(
                    label=class_id,
                    center=Pnt(cx_abs, cy_abs),
                    width=w_abs,
                    height=h_abs,
                    theta=angle,
                    polygon=poly,
                    # id=str(i)
                )
                gt_vehicles.append(gt_vehicle)
    
    return gt_vehicles

def poly2obb_le90_numpy(polys):
    """Convert polygons to oriented bounding boxes - numpy version"""
    polys = polys.reshape(-1, 8)
    pt1, pt2, pt3, pt4 = polys[:, :2], polys[:, 2:4], polys[:, 4:6], polys[:, 6:8]
    
    edge1 = np.sqrt(np.sum((pt1 - pt2)**2, axis=1))
    edge2 = np.sqrt(np.sum((pt2 - pt3)**2, axis=1))
    
    angles1 = np.arctan2(pt2[:, 1] - pt1[:, 1], pt2[:, 0] - pt1[:, 0])
    angles2 = np.arctan2(pt4[:, 1] - pt1[:, 1], pt4[:, 0] - pt1[:, 0])
    
    angles = np.where(edge1 > edge2, angles1, angles2)
    
    x_ctr = (pt1[:, 0] + pt3[:, 0]) / 2.0
    y_ctr = (pt1[:, 1] + pt3[:, 1]) / 2.0
    
    width = np.maximum(edge1, edge2)
    height = np.minimum(edge1, edge2)
    
    return np.column_stack([x_ctr, y_ctr, width, height, angles])

def polygon_overlap_area(poly1: List[List[float]], poly2: List[List[float]]) -> float:
    """Calculate overlap area between two polygons using polygon clipping"""
    try:
        from shapely.geometry import Polygon
        p1 = Polygon(poly1)
        p2 = Polygon(poly2)
        
        if not p1.is_valid:
            p1 = p1.buffer(0)
        if not p2.is_valid:
            p2 = p2.buffer(0)
            
        intersection = p1.intersection(p2)
        return intersection.area if hasattr(intersection, 'area') else 0.0
    except:
        return 0.0

def calculate_iou_rotated_bbox(det: VehicleExport, gt: VehicleExport) -> float:
    """Calculate IoU between two rotated bounding boxes using polygon overlap"""
    overlap = polygon_overlap_area(det.polygon, gt.polygon)
    
    # Calculate areas of both polygons
    try:
        from shapely.geometry import Polygon
        det_poly = Polygon(det.polygon)
        gt_poly = Polygon(gt.polygon)
        
        if not det_poly.is_valid:
            det_poly = det_poly.buffer(0)
        if not gt_poly.is_valid:
            gt_poly = gt_poly.buffer(0)
            
        det_area = det_poly.area
        gt_area = gt_poly.area
        union_area = det_area + gt_area - overlap
        
        return overlap / union_area if union_area > 0 else 0.0
    except:
        return 0.0

def spatial_matching(detections: List[VehicleExport], 
                    ground_truth: List[VehicleExport], 
                    dist_threshold: float = 7.0) -> List[Tuple[int, int, float]]:
    """
    Spatial matching using R-tree index similar to your vehicle matching logic
    Returns list of (det_idx, gt_idx, distance) tuples
    """
    if len(ground_truth) == 0 or len(detections) == 0:
        return []
    
    # Build spatial index for ground truth (like your b_qtree)
    gt_qtree = index.Index()
    for i, gt in enumerate(ground_truth):
        gt_qtree.insert(i, (gt.center.x, gt.center.y))
    
    matches = []
    
    # For each detection, find nearest ground truth (like your matching logic)
    for det_idx, det in enumerate(detections):
        try:
            # Find nearest ground truth vehicle
            nearest_gt_list = list(gt_qtree.nearest((det.center.x, det.center.y), 1, objects=True))
            if not nearest_gt_list:
                continue
                
            gt_idx = nearest_gt_list[0].id
            gt = ground_truth[gt_idx]
            
            # Calculate distance using your euclidean_distance logic
            dist = euclidean_distance(det.center.x, det.center.y, gt.center.x, gt.center.y)
            
            # Apply distance threshold like your code
            if dist <= dist_threshold:
                matches.append((det_idx, gt_idx, dist))
                
        except Exception as e:
            print(f"Error in spatial matching: {e}")
            continue
    
    return matches

def calculate_metrics_with_spatial_matching(detections: List[VehicleExport], 
                                          ground_truth: List[VehicleExport], 
                                          dist_threshold: float = 7.0,
                                          iou_threshold: float = 0.3) -> dict:
    """
    Calculate metrics using spatial matching first, then IoU validation
    This follows your two-stage approach: spatial proximity + validation
    """
    
    if len(ground_truth) == 0:
        if len(detections) == 0:
            return {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'tp': 0, 'fp': 0, 'fn': 0, 'matches': [], 'spatial_candidates': 0}
        else:
            return {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'tp': 0, 'fp': len(detections), 'fn': 0, 'matches': [], 'spatial_candidates': 0}
    
    if len(detections) == 0:
        return {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'tp': 0, 'fp': 0, 'fn': len(ground_truth), 'matches': [], 'spatial_candidates': 0}
    
    # Step 1: Spatial matching (like your qtree nearest neighbor approach)
    spatial_matches = spatial_matching(detections, ground_truth, dist_threshold)
    
    # Step 2: Validate matches with IoU and dimension checks
    validated_matches = []
    matched_gt_indices = set()
    matched_det_indices = set()
    
    # Sort by distance (closest matches first)
    spatial_matches.sort(key=lambda x: x[2])
    
    for det_idx, gt_idx, dist in spatial_matches:
        # Skip if already matched (one-to-one matching)
        if det_idx in matched_det_indices or gt_idx in matched_gt_indices:
            continue
            
        det = detections[det_idx]
        gt = ground_truth[gt_idx]
        
        # Calculate IoU for validation
        iou = calculate_iou_rotated_bbox(det, gt)
        
        # Validate the match (you can add dimension checks here like compare_dimensions)
        if iou >= iou_threshold:
            validated_matches.append((det_idx, gt_idx, dist, iou))
            matched_det_indices.add(det_idx)
            matched_gt_indices.add(gt_idx)
    
    # Calculate metrics
    true_positives = len(validated_matches)
    false_positives = len(detections) - true_positives
    false_negatives = len(ground_truth) - true_positives
    
    precision = true_positives / len(detections) if len(detections) > 0 else 0.0
    recall = true_positives / len(ground_truth) if len(ground_truth) > 0 else 0.0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    
    return {
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'tp': true_positives,
        'fp': false_positives,
        'fn': false_negatives,
        'matches': validated_matches,
        'spatial_candidates': len(spatial_matches)
    }

def benchmark(img_path: str, dets: List[VehicleExport]):
    """
    Benchmark detected vehicles against ground truth using your matching approach
    
    Args:
        img_path: Path to the input image
        dets: List of detected vehicles
    """
    # Load ground truth
    gt_vehicles = load_ground_truth(img_path)
    
    # Use matching parameters similar to your Args class
    dist_thresholds = [5.0, 7.0, 10.0]  # Distance thresholds to test
    iou_thresholds = [0.1, 0.3, 0.5]    # IoU thresholds for validation
    
    print(f"\nBenchmarking {Path(img_path).name}:")
    print(f"Detections: {len(dets)}, Ground Truth: {len(gt_vehicles)}")
    print("-" * 60)
    
    results = {}
    
    for dist_thr in dist_thresholds:
        for iou_thr in iou_thresholds:
            metrics = calculate_metrics_with_spatial_matching(
                dets, gt_vehicles, dist_thr, iou_thr
            )
            
            key = f"dist_{dist_thr}_iou_{iou_thr}"
            results[key] = metrics
            
            print(f"Distance: {dist_thr:4.1f}, IoU: {iou_thr:3.1f} | "
                  f"P: {metrics['precision']:.3f}, "
                  f"R: {metrics['recall']:.3f}, "
                  f"F1: {metrics['f1']:.3f} | "
                  f"TP: {metrics['tp']:2d}, FP: {metrics['fp']:2d}, FN: {metrics['fn']:2d} | "
                  f"Spatial: {metrics['spatial_candidates']:2d}")
    
    # Find best performing combination
    best_f1 = 0
    best_config = None
    for key, metrics in results.items():
        if metrics['f1'] > best_f1:
            best_f1 = metrics['f1']
            best_config = key
    
    if best_config:
        print(f"\nBest configuration: {best_config} with F1: {best_f1:.3f}")
    
    return results

In [99]:
import pathlib as path 
import glob 
dir = "/mnt/d/Dropbox/ZoneSentinel/Challenge/Input"
imgs = txt_files = glob.glob(  f"{dir}/*.png" )
for img_path in imgs:
    args = Args( batch_size=24, patch_size=512, patch_step=256, img=img_path )
    img = mmcv.imread( args.img )
    result = inference_detector_by_patches( model,
                                        img,
                                        args.patch_sizes,
                                        args.patch_steps,
                                        args.img_ratios,
                                        args.merge_iou_thr,
                                        args.batch_size )
    dets = process_results( result )
    benchmark( img_path, dets ) 
    print( dets )

/home/cory/miniconda3/envs/openmmlab-clean/lib/python3.11/site-packages/mmdet/models/dense_heads/anchor_head.py:123: UserWarning: DeprecationWarning: anchor_generator is deprecated, please use "prior_generator" instead
  warnings.warn('DeprecationWarning: anchor_generator is deprecated, '


Detected 28 vehicles.

Benchmarking 1.png:
Detections: 28, Ground Truth: 18
------------------------------------------------------------
Distance:  5.0, IoU: 0.1 | P: 0.643, R: 1.000, F1: 0.783 | TP: 18, FP: 10, FN:  0 | Spatial: 25
Distance:  5.0, IoU: 0.3 | P: 0.536, R: 0.833, F1: 0.652 | TP: 15, FP: 13, FN:  3 | Spatial: 25
Distance:  5.0, IoU: 0.5 | P: 0.214, R: 0.333, F1: 0.261 | TP:  6, FP: 22, FN: 12 | Spatial: 25
Distance:  7.0, IoU: 0.1 | P: 0.643, R: 1.000, F1: 0.783 | TP: 18, FP: 10, FN:  0 | Spatial: 25
Distance:  7.0, IoU: 0.3 | P: 0.536, R: 0.833, F1: 0.652 | TP: 15, FP: 13, FN:  3 | Spatial: 25
Distance:  7.0, IoU: 0.5 | P: 0.214, R: 0.333, F1: 0.261 | TP:  6, FP: 22, FN: 12 | Spatial: 25
Distance: 10.0, IoU: 0.1 | P: 0.643, R: 1.000, F1: 0.783 | TP: 18, FP: 10, FN:  0 | Spatial: 25
Distance: 10.0, IoU: 0.3 | P: 0.536, R: 0.833, F1: 0.652 | TP: 15, FP: 13, FN:  3 | Spatial: 25
Distance: 10.0, IoU: 0.5 | P: 0.214, R: 0.333, F1: 0.261 | TP:  6, FP: 22, FN: 12 | Spatial: 25

/home/cory/miniconda3/envs/openmmlab-clean/lib/python3.11/site-packages/mmdet/models/dense_heads/anchor_head.py:123: UserWarning: DeprecationWarning: anchor_generator is deprecated, please use "prior_generator" instead
  warnings.warn('DeprecationWarning: anchor_generator is deprecated, '


Detected 15 vehicles.

Benchmarking 10.png:
Detections: 15, Ground Truth: 18
------------------------------------------------------------
Distance:  5.0, IoU: 0.1 | P: 0.800, R: 0.667, F1: 0.727 | TP: 12, FP:  3, FN:  6 | Spatial: 13
Distance:  5.0, IoU: 0.3 | P: 0.667, R: 0.556, F1: 0.606 | TP: 10, FP:  5, FN:  8 | Spatial: 13
Distance:  5.0, IoU: 0.5 | P: 0.400, R: 0.333, F1: 0.364 | TP:  6, FP:  9, FN: 12 | Spatial: 13
Distance:  7.0, IoU: 0.1 | P: 0.800, R: 0.667, F1: 0.727 | TP: 12, FP:  3, FN:  6 | Spatial: 13
Distance:  7.0, IoU: 0.3 | P: 0.667, R: 0.556, F1: 0.606 | TP: 10, FP:  5, FN:  8 | Spatial: 13
Distance:  7.0, IoU: 0.5 | P: 0.400, R: 0.333, F1: 0.364 | TP:  6, FP:  9, FN: 12 | Spatial: 13
Distance: 10.0, IoU: 0.1 | P: 0.800, R: 0.667, F1: 0.727 | TP: 12, FP:  3, FN:  6 | Spatial: 13
Distance: 10.0, IoU: 0.3 | P: 0.667, R: 0.556, F1: 0.606 | TP: 10, FP:  5, FN:  8 | Spatial: 13
Distance: 10.0, IoU: 0.5 | P: 0.400, R: 0.333, F1: 0.364 | TP:  6, FP:  9, FN: 12 | Spatial: 1

/home/cory/miniconda3/envs/openmmlab-clean/lib/python3.11/site-packages/mmdet/models/dense_heads/anchor_head.py:123: UserWarning: DeprecationWarning: anchor_generator is deprecated, please use "prior_generator" instead
  warnings.warn('DeprecationWarning: anchor_generator is deprecated, '


...

Detected 22 vehicles.

Benchmarking 7.png:
Detections: 22, Ground Truth: 16
------------------------------------------------------------
Distance:  5.0, IoU: 0.1 | P: 0.682, R: 0.938, F1: 0.789 | TP: 15, FP:  7, FN:  1 | Spatial: 16
Distance:  5.0, IoU: 0.3 | P: 0.591, R: 0.812, F1: 0.684 | TP: 13, FP:  9, FN:  3 | Spatial: 16
Distance:  5.0, IoU: 0.5 | P: 0.409, R: 0.562, F1: 0.474 | TP:  9, FP: 13, FN:  7 | Spatial: 16
Distance:  7.0, IoU: 0.1 | P: 0.682, R: 0.938, F1: 0.789 | TP: 15, FP:  7, FN:  1 | Spatial: 16
Distance:  7.0, IoU: 0.3 | P: 0.591, R: 0.812, F1: 0.684 | TP: 13, FP:  9, FN:  3 | Spatial: 16
Distance:  7.0, IoU: 0.5 | P: 0.409, R: 0.562, F1: 0.474 | TP:  9, FP: 13, FN:  7 | Spatial: 16
Distance: 10.0, IoU: 0.1 | P: 0.682, R: 0.938, F1: 0.789 | TP: 15, FP:  7, FN:  1 | Spatial: 16
Distance: 10.0, IoU: 0.3 | P: 0.591, R: 0.812, F1: 0.684 | TP: 13, FP:  9, FN:  3 | Spatial: 16
Distance: 10.0, IoU: 0.5 | P: 0.409, R: 0.562, F1: 0.474 | TP:  9, FP: 13, FN:  7 | Spatial: 16

/home/cory/miniconda3/envs/openmmlab-clean/lib/python3.11/site-packages/mmdet/models/dense_heads/anchor_head.py:123: UserWarning: DeprecationWarning: anchor_generator is deprecated, please use "prior_generator" instead
  warnings.warn('DeprecationWarning: anchor_generator is deprecated, '


Detected 132 vehicles.

Benchmarking 8.png:
Detections: 132, Ground Truth: 122
------------------------------------------------------------
Distance:  5.0, IoU: 0.1 | P: 0.848, R: 0.918, F1: 0.882 | TP: 112, FP: 20, FN: 10 | Spatial: 124
Distance:  5.0, IoU: 0.3 | P: 0.720, R: 0.779, F1: 0.748 | TP: 95, FP: 37, FN: 27 | Spatial: 124
Distance:  5.0, IoU: 0.5 | P: 0.417, R: 0.451, F1: 0.433 | TP: 55, FP: 77, FN: 67 | Spatial: 124
Distance:  7.0, IoU: 0.1 | P: 0.848, R: 0.918, F1: 0.882 | TP: 112, FP: 20, FN: 10 | Spatial: 124
Distance:  7.0, IoU: 0.3 | P: 0.720, R: 0.779, F1: 0.748 | TP: 95, FP: 37, FN: 27 | Spatial: 124
Distance:  7.0, IoU: 0.5 | P: 0.417, R: 0.451, F1: 0.433 | TP: 55, FP: 77, FN: 67 | Spatial: 124
Distance: 10.0, IoU: 0.1 | P: 0.848, R: 0.918, F1: 0.882 | TP: 112, FP: 20, FN: 10 | Spatial: 124
Distance: 10.0, IoU: 0.3 | P: 0.720, R: 0.779, F1: 0.748 | TP: 95, FP: 37, FN: 27 | Spatial: 124
Distance: 10.0, IoU: 0.5 | P: 0.417, R: 0.451, F1: 0.433 | TP: 55, FP: 77, FN: 67

/home/cory/miniconda3/envs/openmmlab-clean/lib/python3.11/site-packages/mmdet/models/dense_heads/anchor_head.py:123: UserWarning: DeprecationWarning: anchor_generator is deprecated, please use "prior_generator" instead
  warnings.warn('DeprecationWarning: anchor_generator is deprecated, '


Detected 86 vehicles.

Benchmarking 9.png:
Detections: 86, Ground Truth: 86
------------------------------------------------------------
Distance:  5.0, IoU: 0.1 | P: 0.884, R: 0.884, F1: 0.884 | TP: 76, FP: 10, FN: 10 | Spatial: 78
Distance:  5.0, IoU: 0.3 | P: 0.733, R: 0.733, F1: 0.733 | TP: 63, FP: 23, FN: 23 | Spatial: 78
Distance:  5.0, IoU: 0.5 | P: 0.384, R: 0.384, F1: 0.384 | TP: 33, FP: 53, FN: 53 | Spatial: 78
Distance:  7.0, IoU: 0.1 | P: 0.884, R: 0.884, F1: 0.884 | TP: 76, FP: 10, FN: 10 | Spatial: 79
Distance:  7.0, IoU: 0.3 | P: 0.733, R: 0.733, F1: 0.733 | TP: 63, FP: 23, FN: 23 | Spatial: 79
Distance:  7.0, IoU: 0.5 | P: 0.384, R: 0.384, F1: 0.384 | TP: 33, FP: 53, FN: 53 | Spatial: 79
Distance: 10.0, IoU: 0.1 | P: 0.884, R: 0.884, F1: 0.884 | TP: 76, FP: 10, FN: 10 | Spatial: 79
Distance: 10.0, IoU: 0.3 | P: 0.733, R: 0.733, F1: 0.733 | TP: 63, FP: 23, FN: 23 | Spatial: 79
Distance: 10.0, IoU: 0.5 | P: 0.384, R: 0.384, F1: 0.384 | TP: 33, FP: 53, FN: 53 | Spatial: 79